# 控制释放试验 

In [ ]:
import numpy as np

import sys

sys.path.append("C:\\Users\\RS\\VSCode\\matchedfiltermethod\\src")
from algorithms import matchedfilter_methods as mfm

# PRISMA 数据


In [ ]:
prisma_data_filepath = ""
unit_absorption_spectrum_filepath = ""

# 读取 prisma 数据
prisma_datacube = np.load(prisma_data_filepath)

# 读取单位吸收光谱
unit_absorption_spectrum = np.load(unit_absorption_spectrum_filepath)


result = mfm.matched_filter(prisma_datacube, unit_absorption_spectrum,False,False,False)